In [1]:
import keras

Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd

In [3]:
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [4]:
max_features = 20000
maxlen = 100 

In [5]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train = train.sample(frac=1)

In [6]:
list_sentences_train = train["comment_text"].fillna("CVxTz").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").values

In [7]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

In [8]:
def get_model():
    embed_size = 128
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Bidirectional(LSTM(50, return_sequences=True))(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(50, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [9]:
model = get_model()
batch_size = 32
epochs = 2

In [10]:
file_path="weights_base.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [11]:
early = EarlyStopping(monitor="val_loss", mode="min", patience=20)

In [12]:
callbacks_list = [checkpoint, early] #early
model.fit(X_t, y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=callbacks_list)

Train on 86265 samples, validate on 9586 samples
Epoch 1/2
86240/86265 [============================>.] - ETA: 0s - loss: 0.0727 - acc: 0.9772
Epoch 00001: val_loss improved from inf to 0.05118, saving model to weights_base.best.hdf5
86265/86265 [==============================] - 619s 7ms/step - loss: 0.0727 - acc: 0.9772 - val_loss: 0.0512 - val_acc: 0.9817
Epoch 2/2
86240/86265 [============================>.] - ETA: 0s - loss: 0.0472 - acc: 0.9825
Epoch 00002: val_loss improved from 0.05118 to 0.04756, saving model to weights_base.best.hdf5
86265/86265 [==============================] - 627s 7ms/step - loss: 0.0472 - acc: 0.9825 - val_loss: 0.0476 - val_acc: 0.9823


In [13]:
model.load_weights(file_path)
y_test = model.predict(X_te)
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission[list_classes] = y_test
sample_submission.to_csv("baseline.csv", index=False)